# Save Player Cumulative Season BoxScores 

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import time
from nba_api.stats.endpoints import leaguedashplayerstats, leaguegamelog
from tenacity import retry, stop_after_attempt, wait_fixed

box_DIR = "../../data/box/"
shiny_DIR = "../../data/shiny/"

In [3]:
season_start = 2023
season_end = 2025
seasons = np.arange(season_start,season_end,1).astype(str)

league = "NBA"

# Cumulative Boxscores

In [ ]:
def get_box_p_cum(seasons, measure="Base", n=32):
    if measure == "Advanced":
        per_mode = "Per100Possessions"
    else:
        per_mode = "PerGame"
    for season in tqdm(seasons):
        # try:
            season_str = season + "-" + str(int(season) + 1)[-2:]
            stats = leaguedashplayerstats.LeagueDashPlayerStats(
                measure_type_detailed_defense=measure,
                per_mode_detailed=per_mode,
                season=season_str,
            )
            df1 = stats.get_data_frames()[0]
            df = df1.iloc[:, :n]
            if measure == "Advanced":
                measure1 = "Adv"
            else:
                 measure1 = measure
            df.to_parquet(box_DIR + "NBA_Box_P_Cum_" + measure1 + "_" + season + ".parquet")
            time.sleep(0.6)
        # except Exception as error:
        #     continue

In [ ]:
def update_box_p_cum(seasons):
    print("Base")
    get_box_p_cum(seasons, measure="Base", n=32)
    # print("Advanced")
    # get_box_p_cum(seasons, measure="Advanced", n=43)
    # print("Misc")
    # get_box_p_cum(seasons, measure="Misc", n=23)
    # print("Scoring")
    # get_box_p_cum(seasons, measure="Scoring", n=29)

In [ ]:
update_box_p_cum(seasons)

# Individual Game BoxScores

In [ ]:
def update_box_base_p(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [ ]:
update_box_base_p(seasons)

In [4]:
def update_box_base_p_w(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                league_id = "10",
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Regular Season",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + season + ".parquet")
            time.sleep(0.6)
        except Exception as error:
            continue

In [5]:
update_box_base_p_w(seasons)

100%|██████████| 2/2 [00:03<00:00,  1.59s/it]


In [ ]:
dfa = []
for year in range(1946,2024):
    df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    df1 = df1.fillna(0)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.to_parquet(shiny_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [6]:
dfa = []
for year in range(1997,2025):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1[df1["WL"] == 0]["WL"] = "L"
    df1["season"] = year+1
    df1 = df1.fillna(0)
    dfa.append(df1)
df = pd.concat(dfa)
df.loc[df[df["WL"] == 0].index,"WL"] = "L"
df.to_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.to_parquet(shiny_DIR + "WNBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\1919177614.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.fillna(0)
C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\1919177614.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.fillna(0)
C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\1919177614.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option

In [7]:
df = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df.columns = map(str.lower, df.columns)

In [8]:
df = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")
df1 = df[~df.duplicated(keep="last")].reset_index(drop=True)
df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [9]:
df.to_parquet(shiny_DIR + "NBA_Box_P_" + "Base" + "_" + "All" + ".parquet")

In [10]:
pd.options.mode.chained_assignment =  None

In [11]:
cols_i = ['GP', 'W', 'L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 
        'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT','OREB', 'DREB', 'REB', 
        'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS']

In [ ]:
dfa = []
for year in tqdm(range(1946,2024)):
    df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1 = df1.fillna(0)
    df1[df1["WL"] == 0]["WL"] = "L"
    df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_NAME"])
    keys = list(df1g.groups)
    dfb = []
    for key in keys:
        dfi = df1g.get_group(key)
        dfi["GP"] = 1
        dfi["W"] = np.where(dfi["WL"] == "W",1,0)
        dfi["L"] = np.where(dfi["WL"] == "L",1,0)
        dfi1 = dfi[cols_i]
        dfii = pd.DataFrame(dfi1.sum()).T
        dfii.iloc[:,3:] = dfii.iloc[:,3:]/dfii["GP"][0]
        dfii.iloc[:,3:] = dfii.iloc[:,3:].round(2)
        dfii["FG_PCT"] = round(dfii["FGM"]/dfii["FGA"],3)
        dfii["FG3_PCT"] = round(dfii["FG3M"]/dfii["FG3A"],3)
        dfii["FT_PCT"] = round(dfii["FTM"]/dfii["FTA"],3)
        dfii[["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_NAME"]] = list(key)
        dfb.append(dfii)
    dfb = pd.concat(dfb)
    dfb["season"] = year + 1
    dfa.append(dfb)
dfa = pd.concat(dfa)
dfs = dfa.replace([np.inf, -np.inf], np.nan)
dfs = dfs.fillna(0).reset_index(drop=True)
dfs.to_parquet(shiny_DIR+"NBA_Box_P_Cum_Base_All.parquet")

  0%|          | 0/78 [00:00<?, ?it/s]C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\2698142536.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.fillna(0)
  1%|▏         | 1/78 [00:01<01:49,  1.43s/it]C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\2698142536.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.fillna(0)
  3%|▎         | 2/78 [00:02<01:28,  1.16s/it]C:\Users\pansr\AppData\Local\Temp\ipykernel_122140\2698142536.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and wi

In [ ]:
dfa = []
for year in tqdm(range(1997,2024)):
    df1 = pd.read_parquet(box_DIR + "WNBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
    df1 = df1.fillna(0)
    df1[df1["WL"] == 0]["WL"] = "L"
    df1g = df1.groupby(["PLAYER_ID","PLAYER_NAME","TEAM_ID","TEAM_NAME"])
    keys = list(df1g.groups)
    dfb = []
    for key in keys:
        dfi = df1g.get_group(key)
        dfi["GP"] = 1
        dfi["W"] = np.where(dfi["WL"] == "W",1,0)
        dfi["L"] = np.where(dfi["WL"] == "L",1,0)
        dfi1 = dfi[cols_i]
        dfii = pd.DataFrame(dfi1.sum()).T
        dfii.iloc[:,3:] = dfii.iloc[:,3:]/dfii["GP"][0]
        dfii.iloc[:,3:] = dfii.iloc[:,3:].round(2)
        dfii["FG_PCT"] = round(dfii["FGM"]/dfii["FGA"],3)
        dfii["FG3_PCT"] = round(dfii["FG3M"]/dfii["FG3A"],3)
        dfii["FT_PCT"] = round(dfii["FTM"]/dfii["FTA"],3)
        dfii[["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_NAME"]] = list(key)
        dfb.append(dfii)
    dfb = pd.concat(dfb)
    dfb["season"] = year + 1
    dfa.append(dfb)
dfa = pd.concat(dfa)
dfs = dfa.replace([np.inf, -np.inf], np.nan)
dfs = dfs.fillna(0).reset_index(drop=True)
dfs.to_parquet(shiny_DIR+"WNBA_Box_P_Cum_Base_All.parquet")

In [ ]:
dfs.query("PLAYER_NAME == 'LeBron James'")

In [ ]:
from nba_api.stats.endpoints import leagueleaders

In [ ]:
def get_season_leaders(year):
    season_str = str(year) + "-" + str(year+1)[-2:]
    stats = leagueleaders.LeagueLeaders(
        league_id="00",
        per_mode48="PerGame",
        scope="RS",
        season=season_str,
        season_type_all_star="Regular Season",
        stat_category_abbreviation="PTS"
    )
    df = stats.get_data_frames()[0]
    df.to_parquet(box_DIR+ f"NBA_BOX_P_Lead_Base_{year}.parquet")

In [ ]:
for year in range(1946,2024):
    get_season_leaders(year)
    time.sleep(0.6)

In [ ]:
year = 2017
df1 = pd.read_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + str(year) + ".parquet")
len(df1["PLAYER_ID"].unique())

# Playoffs

In [ ]:
# Update BoxScores
def get_gameids(season, name):
    df = pd.read_parquet(
        box_DIR + "NBA_Box_T_" + "Base" + "_" + season + "_PS.parquet"
    )
    game_ids1 = df["GAME_ID"].tolist()
    game_ids1 = np.unique(game_ids1)
    try:
        dfr1 = pd.read_parquet(
            box_DIR + "NBA_Box_T_" + name + "_" + season + "_PS.parquet"
        )
        dfr2 = pd.read_parquet(
            box_DIR + "NBA_Box_P_" + name + "_" + season + "_PS.parquet"
        )
        game_ids3 = dfr1["gameId"].tolist()
        game_ids2 = np.unique(game_ids3)
        game_ids2 = ["00" + str(s) for s in game_ids2]
        game_ids = list(set(game_ids1).difference(game_ids2))
    except Exception as error:
        print(error)
        game_ids = game_ids1
        dfr1 = pd.DataFrame()
        dfr2 = pd.DataFrame()
    return game_ids, dfr1, dfr2

@retry(stop=stop_after_attempt(5), wait=wait_fixed(0.6))
def get_game_box(game_id, fun):
    try:
        t1 = time.perf_counter()
        stats = fun(game_id=game_id)
        df = stats.get_data_frames()
        t2 = time.perf_counter() - t1
        tsleep = 0.6
        if (t2)<0.6:
            time.sleep(tsleep)
    except Exception as error:
        print(error)
        df = pd.DataFrame()
        pass
    return df

def get_games_box(game_ids, fun):
    df_ap1, df_ap2 = [], []
    for game_id in tqdm(game_ids):
        try:
            df0 = get_game_box(game_id, fun)
            df1 = df0[1]
            df2 = df0[0]
        except Exception as error:
            print(error)
            df1 = pd.DataFrame()
            df2 = pd.DataFrame()
        df_ap1.append(df1)
        df_ap2.append(df2)
    return df_ap1, df_ap2


def update_boxscores_idv(season, fun, name):
    game_ids, dfr1, dfr2 = get_gameids(season, name)
    try:
        df_ap1, df_ap2 = get_games_box(game_ids, fun)
        df1 = pd.concat(df_ap1)
        df2 = pd.concat(df_ap2)
        df3 = pd.concat([dfr1, df1])
        df3["gameId"] = df3["gameId"].astype(int)
        df3 = df3.sort_values(by=["gameId"]).reset_index(drop=True)
        df3.to_parquet(
            box_DIR + "NBA_Box_T_" + name + "_" + season + "_PS.parquet"
        )
        df4 = pd.concat([dfr2, df2])
        df4["gameId"] = df4["gameId"].astype(int)
        df4 = df4.sort_values(by=["gameId"]).reset_index(drop=True)
        df4.to_parquet(box_DIR+ "NBA_Box_P_"+ name + "_"+ season+ "_PS.parquet")
    except Exception as error:
        print(error)
        pass

In [ ]:
from nba_api.stats.endpoints import (
    boxscoreadvancedv3,
    boxscorefourfactorsv3,
    boxscorehustlev2,
    boxscoremiscv3,
    boxscoreplayertrackv3,
    boxscorescoringv3,
    leaguegamelog,
    leaguedashplayerstats,
)

In [ ]:
boxscores = [
    {
        "name": "Adv",
        "fun": boxscoreadvancedv3.BoxScoreAdvancedV3,
    },
    {
        "name": "4Factor",
        "fun": boxscorefourfactorsv3.BoxScoreFourFactorsV3,
    },
    {
        "name": "Misc",
        "fun": boxscoremiscv3.BoxScoreMiscV3,
    },
    {
        "name": "Scoring", 
        "fun": boxscorescoringv3.BoxScoreScoringV3
    },
    {
        "name": "Track",
        "fun": boxscoreplayertrackv3.BoxScorePlayerTrackV3,
    },
    {
        "name": "Hustle",
        "fun": boxscorehustlev2.BoxScoreHustleV2,
    },
]

In [ ]:
game_ids, dfr1, dfr2 = get_gameids("2023", "Adv")

In [ ]:
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=2023,
    season_type_all_star="Playoffs",
)
df = stats.get_data_frames()[0]
df["GAME_DATE"] = pd.to_datetime(df["GAME_DATE"], format="%Y-%m-%d")
df.to_parquet(box_DIR + "NBA_Box_T_" + "Base" + "_" + str(2023) + "_PS.parquet")

In [ ]:
season_start = 2023
season_end = 2024
seasons = np.arange(season_start,season_end,1).astype(str)

league = "NBA"

In [ ]:
def update_box_base_p(seasons):
    for season in tqdm(seasons):
        try:
            stats = leaguegamelog.LeagueGameLog(
                player_or_team_abbreviation="P",
                season=season,
                season_type_all_star="Playoffs",
            )
            df = stats.get_data_frames()[0]
            df.to_parquet(box_DIR + "NBA_Box_P_" + "Base" + "_" + season + "_PS.parquet")
            time.sleep(0.6)
        except Exception as error:
            continue
update_box_base_p(seasons)

100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


In [ ]:
for year in range(2023,2024):
    for boxscore in boxscores[:1]:
        print("BoxScore " + boxscore["name"])
        update_boxscores_idv(str(year), boxscore["fun"], boxscore["name"])

In [ ]:
for year in range(1996,2013):
    for boxscore in boxscores[:1]:
        print("BoxScore " + boxscore["name"])
        update_boxscores_idv(str(year), boxscore["fun"], boxscore["name"])

In [ ]:
for year in range(2013,2023):
    for boxscore in boxscores[1:]:
        print("BoxScore " + boxscore["name"])
        update_boxscores_idv(str(year), boxscore["fun"], boxscore["name"])